In [26]:
import ipywidgets as widgets
from IPython.display import display

# Create a dropdown widget for environment selection
interface_dropdown = widgets.Dropdown(
    options=["LAWSON-0005", "LAWSON-0006", "LAWSON-0007"],
    value="LAWSON-0005",
    description="Interface:",
)

#outputdirectory = widgets.Text("testdata")

# Display the widget
# display(widgets.HBox([interface_dropdown, outputdirectory]))
display(widgets.HBox([interface_dropdown]))

### Read Configuration Data

In [27]:
from sqlalchemy import create_engine
import pandas as pd
# Database connection parameters
db_user = 'root'
db_password = 'strong_password'
db_host = 'localhost'
db_name = 'yourdb'
db_port = 33062
interface_id = interface_dropdown.value

# Create a database connection
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Query to read data for a specific interfaceID
queryInterface = f"SELECT * FROM interface WHERE interfaceID = '{interface_id}'"

# Load data into dataframe
dfInterface = pd.read_sql(queryInterface, engine)

mappingId = dfInterface['MappingID'].astype(str).to_string(index=False)

# Query to read data for a specific interfaceID
queryMapping = f"SELECT * FROM mapping WHERE mappingID = '{mappingId}'"

# Load data into dataframe
dfMapping = pd.read_sql(queryMapping, engine)

display(dfInterface)
display(dfMapping)

,InterfaceID,Source,SourceType,SourcePath,Destination,DestinationType,DestinationPath,Program,MappingID,Description
0,LAWSON-0005,File,None,None,Database,None,None,FileToTable,MAPPING-0001,Testing Concept


,MappingID,SourceField,SourceType,Start,SourceLength,Destination,DestinationType,DestinationLength,MappingFunction
0,MAPPING-0001,ID,FIXED,0,4,ID,INT,NaN,PASSTHROUGH
1,MAPPING-0001,Name,FIXED,4,14,Name,VARCHAR,50.0,PASSTHROUGH
2,MAPPING-0001,Date,FIXED,18,8,Date,DATE,NaN,PASSTHROUGH
3,MAPPING-0001,Amount,FIXED,26,8,Amount,FLOAT,2.0,PASSTHROUGH


#### Read Fixed Length File

In [28]:
import pandas as pd

colspecs = [(row['Start'], row['Start'] + row['SourceLength']) for index, row in dfMapping.iterrows()]
names = dfMapping['SourceField'].tolist()  # Column names
dfSource = pd.read_fwf('SourceFileFixedWidth.dat', colspecs=colspecs, names=names, header=None)

print(dfSource)

     ID         Name      Date   Amount
0  1234     John Doe  20231231   100.00
1  5678   Jane Smith  20240115   200.50
2  9012  Peter Jones  20240229    50.25
3  3456   Mary Brown  20240315  4000.75


#### Map to Destination

In [29]:
# Create an empty DataFrame with the same columns as dfSource
dfDestination = pd.DataFrame(columns=dfSource.columns)

def DataMappingAndTransformation(dfDestination, destination, source, destination_type):
    if(destination_type == 'Date'):
        dfDestination[destination] = pd.to_datetime(dfSource[source], format='%Y%m%d', errors='coerce')
    elif(destination_type == 'Numeric'):
        dfDestination[destination] = pd.to_numeric(dfSource[source])
    else:
        dfDestination[destination] = dfSource[source]

for index, row in dfMapping.iterrows():
    destination = row['Destination']
    source = row['SourceField']
    mapping_function = row['MappingFunction']
    destination_type = row['DestinationType']
    DataMappingAndTransformation(dfDestination, destination, source, destination_type)
    
print(dfDestination)

     ID         Name      Date   Amount
0  1234     John Doe  20231231   100.00
1  5678   Jane Smith  20240115   200.50
2  9012  Peter Jones  20240229    50.25
3  3456   Mary Brown  20240315  4000.75


#### Load into Destination